<a href="https://colab.research.google.com/github/nferrucho/NPL/blob/main/proyecto/Amazon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
import pandas as pd

# URL convertida al formato raw
url = 'https://raw.githubusercontent.com/nferrucho/NPL/main/proyecto/amazon_reviews.csv'

# Leer el archivo CSV directamente desde GitHub
df = pd.read_csv(url)

# Mostrar las primeras filas del DataFrame
df.head()


,reviewId,userName,content,score,thumbsUpCount,reviewCreatedVersion,at,appVersion
0,75a98b82-b222-442b-b8e6-dba1dff5eec0,Jason,Would be nice if the search function actually ...,1,3,28.16.0.100,2024-09-08 18:48:32,28.16.0.100
1,cd5e35df-eda2-43e7-a78c-76ae5c8b880a,Rashid Ali,Good,5,0,28.16.0.100,2024-09-08 18:44:39,28.16.0.100
2,62dba633-af93-4fd8-a42b-3d03f085b99d,Octávio Viana,Uma aldrabice o desconto de 10% para compras a...,1,0,NaN,2024-09-08 18:15:12,NaN
3,846f512f-428a-4147-a790-27e20a0af439,Christiaan Burger,Do not use Amazon,1,0,28.13.6.100,2024-09-08 18:01:03,28.13.6.100
4,bebf8fc4-6640-47e6-b3f1-d674c244c309,Diane L,Missing words when you want to go to account s...,3,261,28.16.0.100,2024-09-08 17:46:29,28.16.0.100


In [18]:
datos = df.content
print(datos.head())

0    Would be nice if the search function actually ...
1                                                 Good
2    Uma aldrabice o desconto de 10% para compras a...
3                                    Do not use Amazon
4    Missing words when you want to go to account s...
Name: content, dtype: object


In [65]:
import re

# Función mejorada para normalizar y tokenizar texto
def normalizar_tokenizar(texto):
  try:
      # Eliminar texto vacio
      if pd.isna(texto) or texto.strip() == '':
        return []

      # Expresión regular para encontrar números y porcentajes
      patron_numeros = r'\d+(?:\.\d+)?%'  # Coincide con porcentajes como '45%' o '23.5%'
      patron_palabras_numeros = r'\d+(?:\.\d+)?'  # Coincide con números como '123' o '45.67'

      # Extraer los porcentajes y números del texto antes de aplicar lower
      porcentajes = re.findall(patron_numeros, texto)
      numeros = re.findall(patron_palabras_numeros, texto)

      # Convertir a minúsculas, pero no los números ni los porcentajes
      texto = re.sub(patron_numeros, '', texto)  # Quitar los porcentajes temporariamente
      texto = re.sub(patron_palabras_numeros, '', texto)  # Quitar los números temporariamente
      texto = texto.lower()

      # Volver a insertar los números y porcentajes en el texto
      for p in porcentajes:
          texto += f' {p}'
      for n in numeros:
          texto += f' {n}'

      # Eliminar caracteres especiales, pero mantener los espacios
      texto = re.sub(r'[^\w\s%]', '', texto)

      # Tokenizar (dividir en palabras)
      tokens = texto.split()

      # Filtrar palabras stop
      #tokens_filtrados = [token for token in tokens if token not in stop_words]

      # Normalizacion
      preprocess_text = ' '.join([tokens])
      return tokens

  except Exception as e:
      print(f"Error procesando el texto: {texto}")
      print(f"Error: {e}")
      return []

In [67]:
# Aplicar la función 'normalizar_tokenizar' a la columna 'review_text'
df['review_text_tokenizado'] = df['content'].apply(normalizar_tokenizar)

# Mostrar las primeras filas con la nueva columna
print(df[['review_text_tokenizado']].head())

                              review_text_tokenizado
0  [would, be, nice, if, the, search, function, a...
1                                             [good]
2  [uma, aldrabice, o, desconto, de, para, compra...
3                             [do, not, use, amazon]
4  [missing, words, when, you, want, to, go, to, ...


In [78]:
import spacy

# Cargar el modelo
nlp = spacy.load("en_core_web_sm")
doc = nlp(datos[0])
print(doc)

Would be nice if the search function actually worked correctly. Shows very little of what I actually searched


In [85]:

#TEST_CELL
!pip install unidecode

In [91]:
import pandas as pd
import re
from unidecode import unidecode

def normalizar_tokenizarSpacy(texto):
  try:
    if pd.isna(texto) or texto.strip() == '':
        return []

    # Procesar el texto con spaCy
    doc = nlp(texto)

    # Filtrar tokens usando spaCy
    tokens_filtrados = [
        token.text.lower()
        for token in doc
        if not token.is_stop and token.pos_ in {'NOUN', 'VERB'}
    ]

    return tokens_filtrados
  except Exception as e:
      print(f"Error procesando el texto: {texto}")
      print(f"Error: {e}")
      return []


In [ ]:
# Aplicar la función 'normalizar_tokenizar' a la columna 'content'
df['content_tokenizado'] = df['content'].apply(normalizar_tokenizarSpacy)

# Mostrar las primeras filas con la nueva columna
print(df[['content', 'content_tokenizado']].head())

In [71]:
print(df[['content_tokenizado']].head())

                                  content_tokenizado
0        [search, function, worked, shows, searched]
1                                                 []
2                  [aldrabice, o, %, para, euros, o]
3                                              [use]
4  [missing, words, want, account, settings, hit,...


In [74]:

#TEST_CELL
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 13.6 MB/s eta 0:00:00


In [95]:

def preprocess_word_list(word_list):

    # Unir las palabras en un texto para que spaCy pueda procesarlas como un documento
    text = ' '.join(word_list)

    # Procesar el texto con spaCy
    doc = nlp(text)

    # Lista de etiquetas POS a filtrar (sustantivos y verbos)
    pos_list = {'NOUN', 'VERB'}

    # Filtrar tokens que no sean stopwords y que tengan las etiquetas POS especificadas
    filtered_tokens = [token for token in doc if not token.is_stop and token.pos_ in pos_list]

    # Normalizar y convertir a minúsculas, eliminar acentos
    preprocess_text = ' '.join([unidecode(token.text.lower()) for token in filtered_tokens])

    return preprocess_text

# Ejemplo de uso
word_list = ["Este", "es", "un", "ejemplo", "de", "texto", "preprocesado", "spaCy"]
resultado = preprocess_word_list(word_list)
print(resultado)


este preprocesado spacy


In [96]:
# Aplicar la función 'normalizar_tokenizar' a la columna 'content'
df['content_normalizado'] = df['content_tokenizado'].apply(preprocess_word_list)

# Mostrar las primeras filas con la nueva columna
print(df[['content_normalizado']].head())

                                 content_normalizado
0              search function worked shows searched
1                                                   
2                                aldrabice o euros o
3                                                use
4  words want account settings hit body person sh...
